<a href="https://colab.research.google.com/github/cedro3/stylegan2/blob/master/Find_your_favorite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Find_your_favotite 

# 1.Set up
指定環境である **tensorflow 1.14** に変更し、それに伴い **keras 2.0.8** に変更します。そして、githubからコードをダウンロードします。


In [ ]:
!pip install tensorflow==1.14
!pip install tensorflow-gpu==1.14
!pip install keras==2.0.8

In [ ]:
!git clone https://github.com/cedro3/stylegan2.git

In [ ]:
cd stylegan2

# 2.Difine function
関数を３つ定義します。\
**generate_images** : バッチサイズのベクトルを渡すと画像を表示し、画像は'pic1'フォルダーに上書き保存(最大１００個)。\
**style_mix_example** : ７つのベクトルを渡すと２×５のstyle_mixing結果を表示し、画像は'pic2'フォルダーに上書き保存。\
**generate_gif** : ２つのベクトルを渡すと補完してgifアニメーションを作成し、'gif'フォルダーに上書き保存。




In [ ]:
import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
import pretrained_networks
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tqdm import trange

def generate_images(vectors):
    network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'
    truncation_psi = 0.5  
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = True
    if truncation_psi is not None:
        Gs_kwargs.truncation_psi = truncation_psi

    fig = plt.figure(figsize=(30, 40))
    os.makedirs('pic1', exist_ok=True)    
    for i in trange(len(vectors)):
        images = Gs.run(vectors[i], None, **Gs_kwargs) 
        PIL.Image.fromarray(images[0], 'RGB').save('./pic1/'+str(i)+'.png')     
 
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images[0])
        ax.imshow(image_plt)
        ax.set_xlabel(str(i), fontsize=20)               
    plt.show()
    plt.close()


def style_mixing_example(vectors):  
    network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'
    row_seeds = [1, 2]
    col_seeds = [3, 4, 5, 6, 7]
    col_styles = [0, 1, 2, 3, 4, 5] 
    truncation_psi = 0.5
    minibatch_size = 4

    # Loading networks 
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
    w_avg = Gs.get_var('dlatent_avg') # [component]

    Gs_syn_kwargs = dnnlib.EasyDict()
    Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_syn_kwargs.randomize_noise = False
    Gs_syn_kwargs.minibatch_size = minibatch_size

    # Generating W vectors
    all_seeds = list(row_seeds + col_seeds)   
    all_z = vectors
    all_w = Gs.components.mapping.run(all_z, None) # [minibatch, layer, component]
    all_w = w_avg + (all_w - w_avg) * truncation_psi # [minibatch, layer, component] 
    w_dict = {seed: w for seed, w in zip(all_seeds, list(all_w))} # [layer, component]

    print('Generating images...') 
    all_images = Gs.components.synthesis.run(all_w, **Gs_syn_kwargs) # [minibatch, height, width, channel]
    image_dict = {(seed, seed): image for seed, image in zip(all_seeds, list(all_images))}

    print('Generating style-mixed images...')
    for row_seed in row_seeds:
        for col_seed in col_seeds:
            w = w_dict[row_seed].copy()
            w[col_styles] = w_dict[col_seed][col_styles]
            image = Gs.components.synthesis.run(w[np.newaxis], **Gs_syn_kwargs)[0]
            image_dict[(row_seed, col_seed)] = image

    print('Saving images...')
    os.makedirs('pic2', exist_ok=True)
    for (row_seed, col_seed), image in image_dict.items():
        PIL.Image.fromarray(image, 'RGB').save('./pic2/'+str(row_seed)+'-'+str(col_seed)+'.png')

    print('Saving image grid...')
    _N, _C, H, W = Gs.output_shape
    canvas = PIL.Image.new('RGB', (W * (len(col_seeds) + 1), H * (len(row_seeds) + 1)), 'black')

    r, c = 3, 6  # スクリーン設定（３行×６列）
    fig, axs = plt.subplots(r, c, figsize=(16,8), subplot_kw=({'xticks':(),'yticks':()}))

    for row_idx, row_seed in enumerate([None] + row_seeds):
        for col_idx, col_seed in enumerate([None] + col_seeds):
            if row_seed is None and col_seed is None:
                continue
            key = (row_seed, col_seed)
            if row_seed is None:
                key = (col_seed, col_seed)
            if col_seed is None:
                key = (row_seed, row_seed)
            canvas.paste(PIL.Image.fromarray(image_dict[key], 'RGB'), (W * col_idx, H * row_idx)) 

            # スクリーンに画像配置            
            image_plt = np.array(image_dict[key])
            axs[row_idx, col_idx].imshow(image_plt)
            if row_seed is None:
                x, y = col_seed, col_seed
            elif col_seed is None:
                x, y = row_seed, row_seed
            else:
                x, y = row_seed, col_seed
            axs[row_idx, col_idx].set_xlabel(str(x)+'-'+str(y))

    canvas.save('./pic2/grid.png') 

    # スクリーン表示
    black = np.zeros((1024,1024,3))  # 黒画像作成
    axs[0,0].imshow(black)
    axs[0,0].axis('off')
    plt.show()
    plt.close()   


def generate_gif(vec0,vec1):
    network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'
    truncation_psi = 0.5  # 0.5
    
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = True # False
    if truncation_psi is not None:
        Gs_kwargs.truncation_psi = truncation_psi

    image_gif = []
    image_gif_256 = []
    os.makedirs('gif', exist_ok=True)    
    for i in range(40):
        vec = vec0+(vec1-vec0)*i/39
        images = Gs.run(vec, None, **Gs_kwargs) 
        image_one = PIL.Image.fromarray(images[0], 'RGB')#.save('picture/vec_'+str(i)+'.png')  
        image_gif.append(image_one)
        image_gif_256.append(image_one.resize((256,256))) 

    image_gif[0].save('./gif/movie.gif', save_all=True, append_images=image_gif[1:],
                      duration=100, loop=0)
    image_gif_256[0].save('./gif/movie_256.gif', save_all=True, append_images=image_gif_256[1:],
                      duration=100, loop=0)
    print('complete !')  

# 3.Generate your favorite images
　コードを実行すると、５つのランダムベクトルから画像を生成しますので、好みのベクトルがあれば **plusベクトル**に蓄積し、嫌いなベクトルがあれば **minusベクトル**に蓄積します。すると、**徐々に自分好みの画像が生成される**ようになります。\
　生成した画像は、'pic1'フォルダーに上書き保存(0.png〜4.png)されますので、必要があればその都度ダウンロードして下さい。\
※初回は、pluginの設定があるので処理に時間が掛かりますが、２回目以降は早いです。

In [ ]:
#　初期化
np.random.seed(10) 
vectors = np.zeros((5,512))
plus_vector = np.zeros((1,512))
minus_vector = np.zeros((1,512))

In [ ]:
#@title please, you change the selected action if you need. 
No_0 = 'nothing' #@param ['plus', 'nothing', 'minus']
No_1 = 'nothing' #@param ['plus', 'nothing', 'minus']
No_2 = 'nothing' #@param ['plus', 'nothing', 'minus']
No_3 = 'nothing' #@param ['plus', 'nothing', 'minus']
No_4 = 'nothing' #@param ['plus', 'nothing', 'minus']
vec = [No_0, No_1, No_2, No_3, No_4]

for i in range(len(vectors)):
    if vec[i] == 'plus':
      plus_vector = np.concatenate([plus_vector, vectors[i].reshape([1,512])],axis=0)

    if vec[i] == 'minus':
      minus_vector = np.concatenate([minus_vector, vectors[i].reshape([1,512])], axis=0)

plus_vector_mean = np.mean(plus_vector, axis=0) # plus_vectorの平均をとる
minus_vector_mean = np.mean(minus_vector, axis=0)  # minus_vectorの平均をとる

vectors = np.random.normal(0, 1, (5,1,512))  # ランダムベクトル取得
vectors = vectors + plus_vector_mean - minus_vector_mean  # ベクトルの補正
generate_images(vectors)

print('number of plus_vector = ', len(plus_vector)-1)
print('number of minus_vector = ', len(minus_vector)-1)

# 4.Generate your favorite images using style_mixing
　コードを実行すると、７つのランダムベクトルで、２行×５列のstyle_mixingを行います。５つの行方向のベクトルには、plusベクトルとminusベクトルの補正が効かせてあります。２つの列方向のベクトルには効かせていません。\
　好みの画像に、style_mixingで若干変化を与えることで、さらに良いものを探します。\
　生成した画像(1-1.png 〜 7-7.png & grid.png)は、'pic2'フォルダーに上書き保存されますので、必要があればその都度ダウンロードして下さい。


In [ ]:
vectors_row = np.random.normal(0, 1, (2,512))
vectors_col = np.random.normal(0, 1, (5,512)) + plus_vector_mean - minus_vector_mean
vectors = np.concatenate([vectors_row, vectors_col])
style_mixing_example(vectors)

# 5.Gif animation
　最初のコードを実行すると、１００個のランダムベクトルで画像を生成します(約２分程度掛かります）。その中から２つのベクトルを指定して次のコードを実行すると、ベクトル間を補完したアニメーションを作成します。\
　'pic1'フォルダーに生成した画像 (0.png 〜 99.png) が上書き保存されますので、必要があればその都度ダウンロードして下さい。\
　'gif'フォルダーに、movie.gif (1028×1028サイズ)、movie_256.gif (256×256サイズ)を上書き保存されますので、必要があればその都度ダウンロードして下さい。

In [ ]:
# 100個のベクトルで画像を生成
vectors = np.random.normal(0, 1, (100,1,512))  
generate_images(vectors)  

In [ ]:
# ２つのベクトルを指定し、アニメーションを作成
generate_gif(vectors[43], vectors[50])  